In [24]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
import xgboost as xgb
    

In [13]:
df = pd.read_csv("GLRLM_cyto.csv")
df

,0.0011806,0.0012137,0.001208,0.0012018,1071.2,1046.8,1048.8,1055,1180.7,1552.3,1462.3,1755.1,129.13,156.47,149.55,164.05,0.60938,0.69769,0.66317,0.73551,0.6878,0.73718,0.72884,0.7633,5.531,3.7235,4.3299,3.2685,0.00076172,0.00086877,0.00085176,0.00087827,0.0079293,0.0052889,0.0061704,0.0046729,778.89,795.98,788.88,839.12,4806.8,3260.5,3810,2836.1,-1
0,0.001107,0.001119,0.001116,0.001109,1039.50,1033.20,1037.90,1042.00,2061.90,2842.8,2272.40,2941.2,263.080,315.760,279.550,314.640,1.01060,1.17550,0.68619,1.18670,0.66175,0.72254,0.68387,0.73179,5.4466,3.5958,5.2113,3.5777,0.000721,0.000805,0.000760,0.000802,0.006222,0.004085,0.005960,0.004115,690.72,744.43,709.54,766.11,5627.8,3720.7,5336.2,3671.1,-1
1,0.000838,0.000817,0.000813,0.000816,1474.20,1506.50,1512.10,1509.20,3152.70,4917.1,3974.40,4650.5,502.860,665.500,591.130,649.000,1.33720,1.64370,0.83012,1.59890,0.61612,0.69902,0.66914,0.68952,8.0724,4.8277,7.1607,5.0510,0.000534,0.000582,0.000548,0.000572,0.005821,0.003560,0.005271,0.003754,871.63,1030.40,1000.50,1020.10,13398.0,7915.7,11760.0,8232.1,-1
2,0.000959,0.000957,0.000959,0.000958,1173.50,1181.10,1182.60,1181.20,989.97,1348.5,1125.30,1439.7,181.100,222.810,203.650,227.540,0.63883,0.74565,0.52334,0.76793,0.65301,0.70621,0.67653,0.71998,6.3428,4.1508,5.5416,3.7508,0.000621,0.000669,0.000646,0.000687,0.006186,0.004018,0.005446,0.003669,761.77,836.36,800.78,846.97,7517.2,4942.6,6486.0,4416.6,-1
3,0.001199,0.001197,0.001212,0.001218,998.09,1001.80,989.75,984.42,930.80,1843.0,1685.00,1831.8,142.710,193.000,185.760,197.960,0.37485,1.10080,1.05840,1.11070,0.61956,0.75923,0.74056,0.75332,7.3070,3.4122,3.7966,3.2234,0.000704,0.000868,0.000866,0.000884,0.010184,0.004830,0.005279,0.004436,641.78,785.93,751.50,760.29,6421.8,2951.4,3344.0,2882.6,-1
4,0.001475,0.001478,0.001462,0.001484,907.20,910.03,921.85,900.22,955.55,1267.9,971.25,1291.4,84.396,96.008,85.029,99.075,0.55244,0.62579,0.51733,0.64195,0.70360,0.76095,0.70291,0.75820,6.1534,4.3360,5.7433,3.9587,0.000968,0.001046,0.000953,0.001046,0.012855,0.008977,0.012025,0.008193,665.58,725.56,684.40,712.31,4015.4,2846.9,3709.8,2642.7,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,0.001524,0.001544,0.001554,0.001533,771.82,765.06,760.72,767.49,1757.30,2667.9,2344.80,3036.5,378.780,469.910,441.180,496.620,0.93267,1.11320,0.72514,1.17860,0.55816,0.64131,0.62793,0.66679,9.2472,6.1651,7.9486,5.2714,0.000808,0.000959,0.000969,0.000990,0.015783,0.010407,0.013590,0.009052,449.22,505.74,481.86,525.14,6456.4,4333.4,5510.5,3652.4,-1
912,0.002081,0.002125,0.002143,0.002136,717.41,706.27,692.82,697.34,1442.90,1588.2,1147.20,1742.5,119.020,127.850,116.280,137.210,0.90729,0.94792,0.41054,0.99479,0.77142,0.79092,0.71963,0.80771,4.1068,3.7132,5.3864,3.0782,0.001442,0.001551,0.001455,0.001594,0.011671,0.010588,0.015723,0.008687,605.75,603.61,527.33,604.96,1930.2,1742.7,2547.4,1486.1,-1
913,0.001254,0.001253,0.001270,0.001281,1184.30,1209.10,1198.60,1190.80,2033.90,2822.1,2347.10,2755.8,167.030,207.030,189.770,204.800,0.90604,1.06250,0.71661,1.05730,0.71121,0.77257,0.73634,0.76557,5.3405,3.4361,4.3895,3.3584,0.000853,0.000936,0.000911,0.000946,0.007416,0.005072,0.006287,0.004550,829.15,925.68,873.47,902.69,6960.8,4208.7,5503.9,4354.0,-1
914,0.001481,0.001492,0.001491,0.001518,888.70,883.39,877.48,866.00,1548.80,2111.8,1724.70,2007.8,128.660,150.830,137.660,152.960,0.59731,0.90156,0.82943,0.89297,0.74434,0.80948,0.76450,0.79392,3.8033,2.5448,3.1793,2.5302,0.001054,0.001164,0.001096,0.001173,0.006418,0.004287,0.005301,0.004107,682.09,734.07,688.15,697.72,3022.3,2036.6,2609.3,2124.5,-1


In [14]:
y = df.iloc[:, -1]
y = y.map({-1: 0, 1: 1})
y

0      0
1      0
2      0
3      0
4      0
      ..
911    0
912    0
913    0
914    0
915    0
Name: -1, Length: 916, dtype: int64

In [15]:
y.value_counts()

-1
0    674
1    242
Name: count, dtype: int64

In [16]:
X = df.drop(columns=df.columns[-1])
X

,0.0011806,0.0012137,0.001208,0.0012018,1071.2,1046.8,1048.8,1055,1180.7,1552.3,1462.3,1755.1,129.13,156.47,149.55,164.05,0.60938,0.69769,0.66317,0.73551,0.6878,0.73718,0.72884,0.7633,5.531,3.7235,4.3299,3.2685,0.00076172,0.00086877,0.00085176,0.00087827,0.0079293,0.0052889,0.0061704,0.0046729,778.89,795.98,788.88,839.12,4806.8,3260.5,3810,2836.1
0,0.001107,0.001119,0.001116,0.001109,1039.50,1033.20,1037.90,1042.00,2061.90,2842.8,2272.40,2941.2,263.080,315.760,279.550,314.640,1.01060,1.17550,0.68619,1.18670,0.66175,0.72254,0.68387,0.73179,5.4466,3.5958,5.2113,3.5777,0.000721,0.000805,0.000760,0.000802,0.006222,0.004085,0.005960,0.004115,690.72,744.43,709.54,766.11,5627.8,3720.7,5336.2,3671.1
1,0.000838,0.000817,0.000813,0.000816,1474.20,1506.50,1512.10,1509.20,3152.70,4917.1,3974.40,4650.5,502.860,665.500,591.130,649.000,1.33720,1.64370,0.83012,1.59890,0.61612,0.69902,0.66914,0.68952,8.0724,4.8277,7.1607,5.0510,0.000534,0.000582,0.000548,0.000572,0.005821,0.003560,0.005271,0.003754,871.63,1030.40,1000.50,1020.10,13398.0,7915.7,11760.0,8232.1
2,0.000959,0.000957,0.000959,0.000958,1173.50,1181.10,1182.60,1181.20,989.97,1348.5,1125.30,1439.7,181.100,222.810,203.650,227.540,0.63883,0.74565,0.52334,0.76793,0.65301,0.70621,0.67653,0.71998,6.3428,4.1508,5.5416,3.7508,0.000621,0.000669,0.000646,0.000687,0.006186,0.004018,0.005446,0.003669,761.77,836.36,800.78,846.97,7517.2,4942.6,6486.0,4416.6
3,0.001199,0.001197,0.001212,0.001218,998.09,1001.80,989.75,984.42,930.80,1843.0,1685.00,1831.8,142.710,193.000,185.760,197.960,0.37485,1.10080,1.05840,1.11070,0.61956,0.75923,0.74056,0.75332,7.3070,3.4122,3.7966,3.2234,0.000704,0.000868,0.000866,0.000884,0.010184,0.004830,0.005279,0.004436,641.78,785.93,751.50,760.29,6421.8,2951.4,3344.0,2882.6
4,0.001475,0.001478,0.001462,0.001484,907.20,910.03,921.85,900.22,955.55,1267.9,971.25,1291.4,84.396,96.008,85.029,99.075,0.55244,0.62579,0.51733,0.64195,0.70360,0.76095,0.70291,0.75820,6.1534,4.3360,5.7433,3.9587,0.000968,0.001046,0.000953,0.001046,0.012855,0.008977,0.012025,0.008193,665.58,725.56,684.40,712.31,4015.4,2846.9,3709.8,2642.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,0.001524,0.001544,0.001554,0.001533,771.82,765.06,760.72,767.49,1757.30,2667.9,2344.80,3036.5,378.780,469.910,441.180,496.620,0.93267,1.11320,0.72514,1.17860,0.55816,0.64131,0.62793,0.66679,9.2472,6.1651,7.9486,5.2714,0.000808,0.000959,0.000969,0.000990,0.015783,0.010407,0.013590,0.009052,449.22,505.74,481.86,525.14,6456.4,4333.4,5510.5,3652.4
912,0.002081,0.002125,0.002143,0.002136,717.41,706.27,692.82,697.34,1442.90,1588.2,1147.20,1742.5,119.020,127.850,116.280,137.210,0.90729,0.94792,0.41054,0.99479,0.77142,0.79092,0.71963,0.80771,4.1068,3.7132,5.3864,3.0782,0.001442,0.001551,0.001455,0.001594,0.011671,0.010588,0.015723,0.008687,605.75,603.61,527.33,604.96,1930.2,1742.7,2547.4,1486.1
913,0.001254,0.001253,0.001270,0.001281,1184.30,1209.10,1198.60,1190.80,2033.90,2822.1,2347.10,2755.8,167.030,207.030,189.770,204.800,0.90604,1.06250,0.71661,1.05730,0.71121,0.77257,0.73634,0.76557,5.3405,3.4361,4.3895,3.3584,0.000853,0.000936,0.000911,0.000946,0.007416,0.005072,0.006287,0.004550,829.15,925.68,873.47,902.69,6960.8,4208.7,5503.9,4354.0
914,0.001481,0.001492,0.001491,0.001518,888.70,883.39,877.48,866.00,1548.80,2111.8,1724.70,2007.8,128.660,150.830,137.660,152.960,0.59731,0.90156,0.82943,0.89297,0.74434,0.80948,0.76450,0.79392,3.8033,2.5448,3.1793,2.5302,0.001054,0.001164,0.001096,0.001173,0.006418,0.004287,0.005301,0.004107,682.09,734.07,688.15,697.72,3022.3,2036.6,2609.3,2124.5


In [17]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [30]:
for i, (train_index, test_index) in enumerate(kfold.split(X)):

    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]

    #smote = SMOTE(random_state=42)
    #x_train, y_train = smote.fit_resample(x_train, y_train)

    smote_enn = SMOTEENN(random_state=42)
    x_train, y_train = smote_enn.fit_resample(x_train, y_train)

    model = xgb.XGBClassifier(n_estimators=1000,max_depth=3,colsample_bytree=1.0,learning_rate=0.2,subsample=1.0, scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train))

    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    kappa = cohen_kappa_score(y_test, y_pred)

    print(f'Fold Kappa: {kappa}')

"""
normal

Fold Kappa: 0.5939849624060151
Fold Kappa: 0.6218407248450166
Fold Kappa: 0.5794535705166397
Fold Kappa: 0.7283254156769596
Fold Kappa: 0.5813725490196079

smote

Fold Kappa: 0.7266187050359711
Fold Kappa: 0.5768628328485119
Fold Kappa: 0.5696212797015652
Fold Kappa: 0.6694915254237288
Fold Kappa: 0.5991530373831775

smote_enn

Fold Kappa: 0.631578947368421
Fold Kappa: 0.43423271500843175
Fold Kappa: 0.5459739869182768
Fold Kappa: 0.5221932114882507
Fold Kappa: 0.5574679943100995


"""

Fold Kappa: 0.631578947368421
Fold Kappa: 0.43423271500843175
Fold Kappa: 0.5459739869182768
Fold Kappa: 0.5221932114882507
Fold Kappa: 0.5574679943100995


'\nnormal\n\nFold Kappa: 0.5939849624060151\nFold Kappa: 0.6218407248450166\nFold Kappa: 0.5794535705166397\nFold Kappa: 0.7283254156769596\nFold Kappa: 0.5813725490196079\n\nsmote\n\nFold Kappa: 0.7266187050359711\nFold Kappa: 0.5768628328485119\nFold Kappa: 0.5696212797015652\nFold Kappa: 0.6694915254237288\nFold Kappa: 0.5991530373831775\n\nsmote_enn\n\nFold Kappa: 0.631578947368421\nFold Kappa: 0.43423271500843175\nFold Kappa: 0.5459739869182768\nFold Kappa: 0.5221932114882507\nFold Kappa: 0.5574679943100995\n\n\n'

In [ ]:

# Passo 1: Combine X e y em um único DataFrame (se estiverem separados)
df = pd.concat([X, y], axis=1)
df.columns = list(X.columns) + ['target']  # Renomeie a coluna alvo

# Passo 2: Aplique o SMOTE no DataFrame completo
X_smote, y_smote = SMOTE(random_state=42).fit_resample(df.drop(columns=['target']), df['target'])

# Crie um novo DataFrame balanceado
df_balanced = pd.concat([X_smote, y_smote], axis=1)

# Passo 3: Separe novamente em X e y
X_balanced = df_balanced.drop(columns=['target'])
y_balanced = df_balanced['target']

# Passo 4: Divida em conjuntos de treino e teste
for i, (train_index, test_index) in enumerate(kfold.split(X)):

    x_train, x_test = np.array(X_balanced)[train_index], np.array(X_balanced)[test_index]
    y_train, y_test = np.array(y_balanced)[train_index], np.array(y_balanced)[test_index]

    #smote = SMOTE(random_state=42)
    #x_train, y_train = smote.fit_resample(x_train, y_train)

    model = xgb.XGBClassifier(
        learning_rate=0.1, 
        n_estimators=1000, 
        max_depth=10,
        min_child_weight=1,
        gamma=0,
        subsample=1,
        scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train)
        )

    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    kappa = cohen_kappa_score(y_test, y_pred)

    print(f'Fold Kappa: {kappa}')


"""

learning_rate = None
n_stimators = None

Fold Kappa: 0.6240601503759399
Fold Kappa: 0.5400565504241281
Fold Kappa: 0.5650911011354633
Fold Kappa: 0.7729694227541415
Fold Kappa: 0.5377231391040755

learning_rate = 0.1

Fold Kappa: 0.6642335766423357
Fold Kappa: 0.5400565504241281
Fold Kappa: 0.5775923295454546
Fold Kappa: 0.7373331419549304
Fold Kappa: 0.5685415964971371

learning_rate = 0.1
n_estimators = 1000

Fold Kappa: 0.6541353383458647
Fold Kappa: 0.6049412329095706
Fold Kappa: 0.5723929035049762
Fold Kappa: 0.7729694227541415
Fold Kappa: 0.617698671346202

learning_rate=0.1
n_estimators=1000
max_depth=10

Fold Kappa: 0.6417910447761195
Fold Kappa: 0.546414008155433
Fold Kappa: 0.5903107369929256
Fold Kappa: 0.7875181422351234
Fold Kappa: 0.5932998600707877


learning_rate=0.1, 
n_estimators=1000, 
max_depth=10,
min_child_weight=1,

Fold Kappa: 0.6417910447761195
Fold Kappa: 0.546414008155433
Fold Kappa: 0.5903107369929256
Fold Kappa: 0.7875181422351234
Fold Kappa: 0.5932998600707877


learning_rate=0.1, 
n_estimators=1000, 
max_depth=10,
min_child_weight=1,
gamma=0,

Fold Kappa: 0.6417910447761195
Fold Kappa: 0.546414008155433
Fold Kappa: 0.5903107369929256
Fold Kappa: 0.7875181422351234
Fold Kappa: 0.5932998600707877


learning_rate=0.1, 
n_estimators=1000, 
max_depth=10,
min_child_weight=1,
gamma=0,
subsample=1,

Fold Kappa: 0.6417910447761195
Fold Kappa: 0.546414008155433
Fold Kappa: 0.5903107369929256
Fold Kappa: 0.7875181422351234
Fold Kappa: 0.5932998600707877

"""


Fold Kappa: 0.6417910447761195
Fold Kappa: 0.546414008155433
Fold Kappa: 0.5903107369929256
Fold Kappa: 0.7875181422351234
Fold Kappa: 0.5932998600707877


'\n\nlearning_rate = None\nn_stimators = None\n\nFold Kappa: 0.6240601503759399\nFold Kappa: 0.5400565504241281\nFold Kappa: 0.5650911011354633\nFold Kappa: 0.7729694227541415\nFold Kappa: 0.5377231391040755\n\nlearning_rate = 0.1\n\nFold Kappa: 0.6642335766423357\nFold Kappa: 0.5400565504241281\nFold Kappa: 0.5775923295454546\nFold Kappa: 0.7373331419549304\nFold Kappa: 0.5685415964971371\n\nlearning_rate = 0.1\nn_estimators = 1000\n\nFold Kappa: 0.6541353383458647\nFold Kappa: 0.6049412329095706\nFold Kappa: 0.5723929035049762\nFold Kappa: 0.7729694227541415\nFold Kappa: 0.617698671346202\n\nlearning_rate=0.1\nn_estimators=1000\nmax_depth=10\n\nFold Kappa: 0.6417910447761195\nFold Kappa: 0.546414008155433\nFold Kappa: 0.5903107369929256\nFold Kappa: 0.7875181422351234\nFold Kappa: 0.5932998600707877\n\n\nlearning_rate=0.1, \nn_estimators=1000, \nmax_depth=10,\nmin_child_weight=1,\n\nFold Kappa: 0.6417910447761195\nFold Kappa: 0.546414008155433\nFold Kappa: 0.5903107369929256\nFold Ka

In [ ]:
with open("xgboost_model.pkl", "wb") as file:
    pickle.dump(model, file)